___
## 실습
- tips 데이터에서 grouped 생성
    - groupkey로 'sex', 'day' 지정
- 연속형 변수들(bill, tip 등)은 mean, 범주형 변수는 count를 적용  
___
1. agg
2. apply
    - 필요 시 concat를 사용
3. pivot_table
    - 필요 시 concat 사용
    - pivot_table 사용 시, 중간합계를 계산한다.

In [2]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [3]:
path = 'C:/Users/tmznq/workspace/ml_scratch/seoul_coding_academy/pydata-book-1st-edition/ch08/tips.csv'

tips = pd.read_csv(path)

day,Fri,Sat,Sun,Thur
sex,,,,
Female,9,28,18,32
Male,10,59,58,30


In [108]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [115]:
pd.crosstab(tips['sex'], tips['time'])

time,Dinner,Lunch
sex,,
Female,52,35
Male,124,33


In [ ]:
# sol1) agg, concat

In [123]:
keys = ['sex', 'day']

In [21]:
grouped = tips.groupby(keys)

In [88]:
means = ['total_bill', 'tip', 'size']
cats = ['time', 'smoker']

In [97]:
means1 = tips.groupby(keys)[means].agg('mean')

In [130]:
cats1 = tips.groupby(keys)[cats].agg('count')
cats1

time  smoker
sex    day               
Female Fri      9       9
       Sat     28      28
       Sun     18      18
       Thur    32      32
Male   Fri     10      10
       Sat     59      59
       Sun     58      58
       Thur    30      30

In [91]:
pd.concat([means1, cats1], axis = 1)

total_bill       tip      size  time  smoker
sex    day                                               
Female Fri    14.145556  2.781111  2.111111     9       9
       Sat    19.680357  2.801786  2.250000    28      28
       Sun    19.872222  3.367222  2.944444    18      18
       Thur   16.715312  2.575625  2.468750    32      32
Male   Fri    19.857000  2.693000  2.100000    10      10
       Sat    20.802542  3.083898  2.644068    59      59
       Sun    21.887241  3.220345  2.810345    58      58
       Thur   18.714667  2.980333  2.433333    30      30

In [ ]:
tips.groupby('smoker')['tip_pct'].agg(['count', 'mean', 'std', 'min', 'max'])

In [ ]:
# sol)2 agg + dict

In [84]:
means2 = ['total_bill', 'tip', 'size']
cats2 = ['time', 'smoker']

In [87]:
keys = ['sex', 'day']
tips.groupby(keys).agg({'total_bill': 'mean',
                        'tip': 'mean',
                        'size': 'mean',
                        'time': 'count',
                        'smoker': 'count'})

total_bill       tip      size  time  smoker
sex    day                                               
Female Fri    14.145556  2.781111  2.111111     9       9
       Sat    19.680357  2.801786  2.250000    28      28
       Sun    19.872222  3.367222  2.944444    18      18
       Thur   16.715312  2.575625  2.468750    32      32
Male   Fri    19.857000  2.693000  2.100000    10      10
       Sat    20.802542  3.083898  2.644068    59      59
       Sun    21.887241  3.220345  2.810345    58      58
       Thur   18.714667  2.980333  2.433333    30      30

In [129]:
# sol)3 pivot
means3 = pd.pivot_table(tips, index = keys, values=means, aggfunc='mean')
means3

size       tip  total_bill
sex    day                                 
Female Fri   2.111111  2.781111   14.145556
       Sat   2.250000  2.801786   19.680357
       Sun   2.944444  3.367222   19.872222
       Thur  2.468750  2.575625   16.715312
Male   Fri   2.100000  2.693000   19.857000
       Sat   2.644068  3.083898   20.802542
       Sun   2.810345  3.220345   21.887241
       Thur  2.433333  2.980333   18.714667

In [128]:
cats3 = pd.pivot_table(tips, index = keys, values=cats, aggfunc='count')
cats3

smoker  time
sex    day               
Female Fri        9     9
       Sat       28    28
       Sun       18    18
       Thur      32    32
Male   Fri       10    10
       Sat       59    59
       Sun       58    58
       Thur      30    30

In [94]:
pd.concat([means3, cats3], axis = 1)

size       tip  total_bill  smoker  time
sex    day                                               
Female Fri   2.111111  2.781111   14.145556       9     9
       Sat   2.250000  2.801786   19.680357      28    28
       Sun   2.944444  3.367222   19.872222      18    18
       Thur  2.468750  2.575625   16.715312      32    32
Male   Fri   2.100000  2.693000   19.857000      10    10
       Sat   2.644068  3.083898   20.802542      59    59
       Sun   2.810345  3.220345   21.887241      58    58
       Thur  2.433333  2.980333   18.714667      30    30

In [54]:
# sol)4 apply
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

### 교수님 답
- mean, count의 dict 만드는 for문 참고할 것.
- 이름 지어서 넣어주는 것도 참고할 것.
- grouped.apply(mean_count)